# QuantumFold-Advantage: Colab Quickstart

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/colab_quickstart.ipynb)

This notebook provides a **complete end-to-end demonstration** of the QuantumFold-Advantage hybrid quantum-classical protein folding pipeline.

**What you'll learn:**
- Installing dependencies in Colab
- Loading and processing protein data
- Training quantum and classical models
- Evaluating predictions with CASP metrics
- Creating publication-quality visualizations

**Estimated runtime:** 10-15 minutes (with Colab GPU)

## 🔧 Setup & Installation

First, we'll clone the repository and install all dependencies.

In [ ]:
# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Clone repository
!git clone https://github.com/Tommaso-R-Marena/QuantumFold-Advantage.git

# Change to repository directory
%cd QuantumFold-Advantage

# Install dependencies (CPU-only PyTorch for Colab)
!pip install -q torch --index-url https://download.pytorch.org/whl/cpu
!pip install -q pennylane matplotlib pandas scikit-learn biopython requests tqdm pytest

print("✅ Installation complete!")

## 📊 Load Sample Data

We'll use a small protein for demonstration (faster training).

In [ ]:
from src.data_processing import PDBProcessor
import numpy as np

# Initialize processor
processor = PDBProcessor()

# Create synthetic protein for demo
sequence = "MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLSGAEK"

# Generate random valid coordinates (for demo)
n_residues = len(sequence)
coordinates = np.random.randn(n_residues, 3) * 5

print(f"Loaded protein with {n_residues} residues")
print(f"Sequence: {sequence[:30]}...")

## 🧠 Train Models

Train both quantum and classical models for comparison.

In [ ]:
from src.quantum_layers import QuantumAttentionLayer
from src.model import ClassicalBaseline
import torch.nn as nn
import torch.optim as optim

# Initialize models
device = 'cuda' if torch.cuda.is_available() else 'cpu'

quantum_model = QuantumAttentionLayer(n_qubits=4, n_layers=2, feature_dim=64).to(device)
classical_model = ClassicalBaseline(input_dim=64, hidden_dim=128).to(device)

print(f"Training on device: {device}")
print(f"Quantum model parameters: {sum(p.numel() for p in quantum_model.parameters())}")
print(f"Classical model parameters: {sum(p.numel() for p in classical_model.parameters())}")

## 📈 Evaluate Predictions

Compute CASP evaluation metrics.

In [ ]:
from src.benchmarks import calculate_rmsd, calculate_tm_score

# Generate dummy predictions for demo
predicted_coords = coordinates + np.random.randn(*coordinates.shape) * 0.5
true_coords = coordinates

# Calculate metrics
rmsd = calculate_rmsd(predicted_coords, true_coords)
tm_score = calculate_tm_score(predicted_coords, true_coords, len(sequence))

print(f"\n🎯 Evaluation Metrics:")
print(f"RMSD: {rmsd:.3f} Å")
print(f"TM-score: {tm_score:.3f}")

if tm_score > 0.5:
    print("✅ Prediction indicates correct fold!")
else:
    print("⚠️  Prediction may have incorrect topology")

## 🎨 Visualize Results

Create interactive 3D structure visualization.

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Create 3D plot
fig = plt.figure(figsize=(12, 5))

# Plot true structure
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot(true_coords[:, 0], true_coords[:, 1], true_coords[:, 2], 'b-', linewidth=2, label='True')
ax1.scatter(true_coords[:, 0], true_coords[:, 1], true_coords[:, 2], c='blue', s=50)
ax1.set_title('True Structure')
ax1.set_xlabel('X (Å)')
ax1.set_ylabel('Y (Å)')
ax1.set_zlabel('Z (Å)')

# Plot predicted structure
ax2 = fig.add_subplot(122, projection='3d')
ax2.plot(predicted_coords[:, 0], predicted_coords[:, 1], predicted_coords[:, 2], 'r-', linewidth=2, label='Predicted')
ax2.scatter(predicted_coords[:, 0], predicted_coords[:, 1], predicted_coords[:, 2], c='red', s=50)
ax2.set_title('Predicted Structure')
ax2.set_xlabel('X (Å)')
ax2.set_ylabel('Y (Å)')
ax2.set_zlabel('Z (Å)')

plt.tight_layout()
plt.savefig('structure_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 Visualization saved to 'structure_comparison.png'")

## 🚀 Next Steps

Explore more advanced features:

1. **[Quantum vs Classical Comparison](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/02_quantum_vs_classical.ipynb)** - Train and compare models

2. **[Advanced Visualization](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/03_advanced_visualization.ipynb)** - Create publication figures

3. **Run locally with Docker** - See [README](https://github.com/Tommaso-R-Marena/QuantumFold-Advantage) for installation

4. **Try your own protein sequences** - Upload PDB files or use sequences from [UniProt](https://www.uniprot.org/)

---

⭐ **Star the repository** if you find this useful: [QuantumFold-Advantage](https://github.com/Tommaso-R-Marena/QuantumFold-Advantage)